In [56]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sys
from scipy.spatial.transform import Rotation as R
sys.path.append('/home/bioinlab/Documents/real-OMX-bioin-tacto-grasp-and-lift/src/RL_grasp_and_lift/scripts')
# Configuration paramaters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 32  # Size of batch taken from replay buffer
max_steps_per_episode = 100

# Use the Baseline Atari environment because of Deepmind helper functions



num_actions = 8


def create_q_model():
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=(15))

    # Convolutions on the frames on the screen
    layer1 = layers.Dense(64, activation="relu")(inputs)
    layer2 = layers.Dense(32, activation="relu")(layer1)
    layer3 = layers.Dense(16, activation="relu")(layer2)

    layer4 = layers.Flatten()(layer3)

    # layer5 = layers.Dense(512, activation="relu")(layer4)
    layer5 = layers.Dense(num_actions, activation="linear")(layer4) #change 
    action = layers.Dense(num_actions, activation="softmax")(layer5) #change 

    #add a softmax layer
    return keras.Model(inputs=inputs, outputs=action)


# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_q_model()
# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = create_q_model()

import pandas as pd
import numpy as np
from imitation.data.types import TrajectoryWithRew
from imitation.data.types import DictObs
traj_index = ['Object2/tf/robot_O2_T1_A0_2024-01-12-11-15-39_withtf',
              'Object2/tf/robot_O2_T3_A0_2024-01-12-11-48-22_withtf',
              'Object2/tf/robot_O2_T4_A0_2024-01-09-12-37-55_withtf',
              #'Object1/tf/try1_final',
              #'Object1/tf/try1_final',
              #'Object1/tf/try1_final',
              #'Object4/tf/robot_O4_T1_A0_2024-01-10-17-03-57_withtf',
              #'Object4/tf/robot_O4_T2_A0_2024-01-10-17-07-43_withtf',
              #'Object4/tf/robot_O4_T3_A0_2024-01-10-17-13-06_withtf'
              ]
dfs_all = [] 
expert_trajectories = []
action = np.array([])  
observation = np.empty(shape=(0,15))  
def shortest_angle_difference(r1, r2):

    r_diff = r2 * r1.inv()  # Rotation representing the shortest rotation from q1 to q2
    euler_angles = r_diff.as_euler('xyz', degrees=True)
    return (euler_angles)

for i in traj_index:
    path = f'/home/bioinlab/Documents/real-OMX-bioin-tacto-grasp-and-lift/bags/{i}.csv'
    df1 = pd.read_csv(path)
    df1 = df1[~df1['instruction'].str.contains('f')]
    df1 = df1[~df1['instruction'].str.contains('g')]

    df1['reward'] = pd.Series(dtype='double')
    df1['reward_sum'] = pd.Series(dtype='double')
    
    df1['action'] = pd.Series(dtype='int')
    
    df1['delta_x_left'] = pd.Series(dtype='double')
    df1['delta_y_left'] = pd.Series(dtype='double')
    df1['delta_z_left'] = pd.Series(dtype='double')
    df1['delta_w_left'] = pd.Series(dtype='double')

    df1['delta_x_right'] = pd.Series(dtype='double')
    df1['delta_y_right'] = pd.Series(dtype='double')
    df1['delta_z_right'] = pd.Series(dtype='double')
    df1['delta_w_right'] = pd.Series(dtype='double')

    df1['episode_start'] = 0
    df1.loc[1,'episode_start'] =  1
    reward_sum = 0
    df1['baro1'] =df1['baro1']/500
    df1['baro2'] =df1['baro2']/500
    #df1 = df1[df1['baro1'] <= 1]
    #df1 = df1[df1['baro2'] <= 1]
    df1 = df1.reset_index(drop=True)
    actions = ['w', 's','a','d','z','x','o','k']
    for row in range(df1.shape[0]):
        if row ==0:
            delta_quart = 0
            df1.loc[row, 'delta_x_left'] = 0
            df1.loc[row, 'delta_y_left'] = 0
            df1.loc[row, 'delta_z_left'] = 0
            df1.loc[row, 'delta_w_left'] = 1
            df1.loc[row, 'delta_x_right'] = 0
            df1.loc[row, 'delta_y_right'] = 0
            df1.loc[row, 'delta_z_right'] = 0
            df1.loc[row, 'delta_w_right'] = 1
        else:
            angle_left_previous = R.from_quat([df1['imu_left_orientation_x'][row-1], 
                                          df1['imu_left_orientation_y'][row-1],
                                          df1['imu_left_orientation_z'][row-1],
                                          df1['imu_left_orientation_w'][row-1]])#.as_euler('zyx', degrees=True)
            angle_right_previous = R.from_quat([df1['imu_right_orientation_x'][row-1], 
                                          df1['imu_right_orientation_y'][row-1],
                                          df1['imu_right_orientation_z'][row-1],
                                          df1['imu_right_orientation_w'][row-1]])#.as_euler('zyx', degrees=True)    
                    
            current_left_angle = R.from_quat([df1['imu_left_orientation_x'][row], 
                                          df1['imu_left_orientation_y'][row],
                                          df1['imu_left_orientation_z'][row],
                                          df1['imu_left_orientation_w'][row]])#.as_euler('zyx', degrees=True)
            current_right_angle = R.from_quat([df1['imu_right_orientation_x'][row], 
                                          df1['imu_right_orientation_y'][row],
                                          df1['imu_right_orientation_z'][row],
                                          df1['imu_right_orientation_w'][row]])#.as_euler('zyx', degrees=True)          
            left_angle_diff = shortest_angle_difference(angle_left_previous, current_left_angle)
            right_angle_diff = shortest_angle_difference(angle_right_previous, current_right_angle)


            #print("left: ", left_angle_diff)
            #print("right: ", current_right_angle)
            df1.loc[row, 'delta_x_left'] = left_angle_diff[0]#* (18 / np.pi)
            df1.loc[row, 'delta_y_left'] = left_angle_diff[1]#* (18 / np.pi)
            df1.loc[row, 'delta_z_left'] = left_angle_diff[2] #* (18 / np.pi)
            df1.loc[row, 'delta_x_right'] =right_angle_diff[0]# * (18 / np.pi)
            df1.loc[row, 'delta_y_right'] =right_angle_diff[1]# * (18 / np.pi)
            df1.loc[row, 'delta_z_right'] =right_angle_diff[2]# * (18 / np.pi)
            
        reward = 0
        #print(df1['baro1'][row])
        if(df1['baro1'][row] < 0.1 or df1['baro2'][row] < 0.1):
            reward = reward - 0.5
        elif (df1['baro1'][row] > 1 or df1['baro2'][row] >1 ):
            reward = reward - 0.5
        if(df1['end-effector_position_z'][row] > 0.11):
            reward = reward + 0.8
        reward = df1['end-effector_position_z'][row] * 3 - 0.1
        reward_sum = reward_sum + reward
        df1.loc[row, 'reward'] = reward
        df1.loc[row,'reward_sum'] = reward_sum
        df1.loc[row,'action'] = actions.index(df1['instruction'][row])   
    dfs_all.append(df1.copy(deep=True))
for i in dfs_all:
    df_all = i
    action = np.append(action, df_all['action'].to_numpy())
    #action = np.delete(action, (0), axis=0)

    print(len(action))
    observation = np.append(observation, df_all[["baro1", "baro2",
                        "delta_x_left","delta_y_left","delta_z_left",
                        "delta_x_right","delta_y_right","delta_z_right",
                      #  "imu_left_orientation_x","imu_left_orientation_y","imu_left_orientation_z","imu_left_orientation_w",
                      #  "imu_right_orientation_x", "imu_right_orientation_y", "imu_right_orientation_z", "imu_right_orientation_w",
    "end-effector_position_x", "end-effector_position_y", "end-effector_position_z",
    "end-effector_orientation_x", "end-effector_orientation_y", "end-effector_orientation_z", "end-effector_orientation_w"]].to_numpy(),axis=0)
# Extract each obs component into its own array
    print(observation.shape)

    
    

18
(18, 15)
36
(36, 15)
54
(54, 15)


In [144]:
dfs_all[0][['delta_x_left','delta_y_left','delta_z_left', 'delta_w_left']]

,delta_x_left,delta_y_left,delta_z_left,delta_w_left
0,0.000000,0.000000,0.000000,1.0
1,0.181060,-0.310753,-0.157860,NaN
2,-0.105481,0.455116,0.335578,NaN
3,6.041812,-4.728385,0.265014,NaN
4,-6.957873,5.173186,0.048128,NaN
5,5.464645,-1.326787,1.162338,NaN
6,-354.854929,-5.094939,1.572630,NaN
7,352.031074,6.122194,-2.766215,NaN
8,2.705911,-0.425595,1.944645,NaN
9,0.823962,-0.640565,0.867140,NaN


In [57]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

one_hot_targets = np.eye(num_actions)[action.astype(int)]


model.fit(observation, one_hot_targets, epochs=400, batch_size=50)


#actions = ['w', 's','a','d','z','x','f','g','o','k']


Epoch 1/400


2/2 [==============================] - 0s 2ms/step - loss: 0.7498 - accuracy: 0.1667
Epoch 2/400
2/2 [==============================] - 0s 2ms/step - loss: 0.7192 - accuracy: 0.1111
Epoch 3/400
2/2 [==============================] - 0s 2ms/step - loss: 0.6935 - accuracy: 0.0926
Epoch 4/400
2/2 [==============================] - 0s 3ms/step - loss: 0.6754 - accuracy: 0.0556
Epoch 5/400
2/2 [==============================] - 0s 2ms/step - loss: 0.6636 - accuracy: 0.1296
Epoch 6/400
2/2 [==============================] - 0s 2ms/step - loss: 0.6560 - accuracy: 0.1667
Epoch 7/400
2/2 [==============================] - 0s 2ms/step - loss: 0.6486 - accuracy: 0.1111
Epoch 8/400
2/2 [==============================] - 0s 1ms/step - loss: 0.6409 - accuracy: 0.1111
Epoch 9/400
2/2 [==============================] - 0s 2ms/step - loss: 0.6328 - accuracy: 0.1111
Epoch 10/400
2/2 [==============================] - 0s 2ms/step - loss: 0.6243 - accuracy: 0.1111
Epoch 11/400
2/2 [=======================

In [24]:
action.astype(int)

array([7, 1, 4, 1, 1, 4, 1, 1, 1, 4, 3, 3, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1,
       1, 1, 1, 4, 1, 1, 4, 1, 1, 1, 1, 1, 4, 1, 5, 1, 5, 8, 9, 4, 4, 1,
       9, 8, 8, 1, 1, 1, 1, 1, 4, 1, 5, 1, 5, 8, 9, 4, 4, 1, 9, 8, 8, 1,
       1, 1, 1, 1, 4, 1, 5, 1, 5, 8, 9, 4, 4, 1, 9, 8, 8, 1, 1, 1, 1, 1,
       1, 4, 1, 1, 4, 4, 1, 1, 3, 3, 1, 1, 1, 4, 4, 1, 1, 1, 4, 3, 3, 1,
       1, 8, 9, 9, 1, 1, 9, 4, 4, 5, 9, 8, 4, 1, 1, 0, 1, 1, 1, 1])

In [58]:
model.save("/home/bioinlab/Documents/real-OMX-bioin-tacto-grasp-and-lift/src/RL_grasp_and_lift/scripts/trained_04_A0.keras")

In [ ]:
new_model = tf.keras.models.load_model('/home/bioinlab/Documents/real-OMX-bioin-tacto-grasp-and-lift/src/RL_grasp_and_lift/scripts/trained_O3_A0.h5')
new_model.summary()


In [ ]:
np.argmax((model.predict(observation)),1)
# action = tf.argmax(action_probs[0]).numpy()

In [ ]:
# In the Deepmind paper they use RMSProp however then Adam optimizer
# improves training time
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0
# Number of frames to take random action and observe output
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 100000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000
# Using huber loss for stability
loss_function = keras.losses.Huber()
pretraining_data = 500 
pretraining_count = 0
while pretraining_count < pretraining_data:  # Run until solved
    state = np.array(env.reset())
    episode_reward = 0
    pretraining_count += 1
    for timestep in range(1, max_steps_per_episode):
        # env.render(); Adding this line would show the attempts
        # of the agent in a pop up window.
        frame_count += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done, _ = env.step(action)
        state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = model_target.predict(stanum_actionste_next_sample)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if running_reward > 40:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break
